In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from pprint import PrettyPrinter

from tensorflow.python.keras.datasets import mnist
from tensorflow.python.keras.models import Model
from tensorflow.python.keras.layers import Input, Dense, Dropout, Conv2D, MaxPool2D, Reshape, Concatenate, Flatten

%matplotlib inline

pp = PrettyPrinter()

In [2]:
from tensorflow.python.keras.preprocessing.text import Tokenizer, one_hot
from tensorflow.python.keras.preprocessing.sequence import pad_sequences

def one_hot_encode(text_index_lists, max_n):
    texts = []
    for idx1, text_index_list in enumerate(text_index_lists):
        text = np.zeros((len(text_index_list), max_n))
        for idx2, index in enumerate(text_index_list):
            text[idx2, index - 1] = 1
        texts.append(text)
    return texts

# dataset
ds_x = [
    'he is a good man',
    'you are a smart woman',
    'she is a bad woman',
    'a bad man',
    'an idiot man he is',
    'is he an idiot',
    'he is an idiot man',
    'you are a good man',
    'you are a smart man',
    'she is a good woman',
    'you are bad',
    'he is good',
    'she is an idiot woman',
    'he is a smart man'
]
# 1: good, 0: bad
ds_y = [1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0 ,1]

# fit tokenizer to get list of indexes to one-hot encode and act as word embeddings
tokenizer = Tokenizer()
tokenizer.fit_on_texts(ds_x)

#max length of word embeddings
embeddings_l = len(tokenizer.word_counts)
print('Length of word embeddings: %d' % embeddings_l, '\n\n')

# list of indexes
list_of_indexes = tokenizer.texts_to_sequences(ds_x)
print('list of indexes')
pp.pprint(list_of_indexes)

# one-hot encode
ds_X = one_hot_encode(list_of_indexes, embeddings_l)
print('One hot encoded')
pp.pprint(ds_X)

#pad sequences for max length of sentence
ds_X = pad_sequences(ds_X, padding='post')
print('After padding')
pp.pprint(ds_X)

max_sequence_length = len(ds_X[0])


Length of word embeddings: 13 


list of indexes
[[4, 1, 2, 5, 3],
 [6, 7, 2, 11, 8],
 [12, 1, 2, 13, 8],
 [2, 13, 3],
 [9, 10, 3, 4, 1],
 [1, 4, 9, 10],
 [4, 1, 9, 10, 3],
 [6, 7, 2, 5, 3],
 [6, 7, 2, 11, 3],
 [12, 1, 2, 5, 8],
 [6, 7, 13],
 [4, 1, 5],
 [12, 1, 9, 10, 8],
 [4, 1, 2, 11, 3]]
One hot encoded
[array([[0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]]),
 array([[0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.]]),
 array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
       [1., 0

In [3]:
X = np.array(ds_X)
X = X.reshape(-1, X.shape[-2], X.shape[-1], 1)
y = np.array(ds_y)
X_train = X[:-4]
y_train = y[:-4]
X_test = X[-4:]
y_test = y[-4:]
max_sequence_length, embeddings_length = X_train.shape[1:3]
print('X_train shape: ', X_train.shape)
print('y_train shape: ', y_train.shape)
print('X_test shape: ', X_test.shape)
print('y_test shape: ', y_test.shape)

X_train shape:  (10, 5, 13, 1)
y_train shape:  (10,)
X_test shape:  (4, 5, 13, 1)
y_test shape:  (4,)


In [4]:
def n_grams_channel(n):
    def n_gram_channel_block(inputs):
        channel = Conv2D(1, kernel_size=(n, embeddings_length), activation='relu')(inputs)
        channel_mp = MaxPool2D(pool_size=(channel.shape[1], 1))(channel)
        channel_final = Flatten()(channel_mp)        
        return channel_final
    return n_gram_channel_block

In [5]:
inputs = Input(shape=(max_sequence_length, embeddings_length, 1))
channel1_final = n_grams_channel(2)(inputs)
channel2_final = n_grams_channel(3)(inputs)
channel3_final = n_grams_channel(4)(inputs)
channels_final = Concatenate()([channel1_final, channel2_final, channel3_final])
predictions = Dense(1, 'sigmoid')(channel1_final)

model = Model(inputs=inputs, outputs=predictions)

model.compile(
    loss='binary_crossentropy',
    optimizer='adam'
)

In [6]:
model.fit(
    X_train, y_train,
    epochs=5000,
)

Epoch 1/5000
10/10 [==============================]10/10 [==============================] - 0s 14ms/step - loss: 0.6970

Epoch 2/5000
10/10 [==============================]10/10 [==============================] - 0s 198us/step - loss: 0.6961

Epoch 3/5000
10/10 [==============================]10/10 [==============================] - 0s 200us/step - loss: 0.6953

Epoch 4/5000
10/10 [==============================]10/10 [==============================] - 0s 200us/step - loss: 0.6945

Epoch 5/5000
10/10 [==============================]10/10 [==============================] - 0s 200us/step - loss: 0.6937

Epoch 6/5000
10/10 [==============================]10/10 [==============================] - 0s 300us/step - loss: 0.6928

Epoch 7/5000
10/10 [==============================]10/10 [==============================] - 0s 401us/step - loss: 0.6920

Epoch 8/5000
10/10 [==============================]10/10 [==============================] - 0s 701us/step - loss: 0.6912

Epoch 9/5000
10/10 [=====


Epoch 68/5000
10/10 [==============================]10/10 [==============================] - 0s 300us/step - loss: 0.6457

Epoch 69/5000
10/10 [==============================]10/10 [==============================] - 0s 200us/step - loss: 0.6449

Epoch 70/5000
10/10 [==============================]10/10 [==============================] - 0s 200us/step - loss: 0.6441

Epoch 71/5000
10/10 [==============================]10/10 [==============================] - 0s 200us/step - loss: 0.6433

Epoch 72/5000
10/10 [==============================]10/10 [==============================] - 0s 200us/step - loss: 0.6425

Epoch 73/5000
10/10 [==============================]10/10 [==============================] - 0s 499us/step - loss: 0.6417

Epoch 74/5000
10/10 [==============================]10/10 [==============================] - 0s 500us/step - loss: 0.6409

Epoch 75/5000
10/10 [==============================]10/10 [==============================] - 0s 301us/step - loss: 0.6400

Epoch 76/5000
1


Epoch 135/5000
10/10 [==============================]10/10 [==============================] - 0s 300us/step - loss: 0.5869

Epoch 136/5000
10/10 [==============================]10/10 [==============================] - 0s 901us/step - loss: 0.5860

Epoch 137/5000
10/10 [==============================]10/10 [==============================] - 0s 200us/step - loss: 0.5851

Epoch 138/5000
10/10 [==============================]10/10 [==============================] - 0s 200us/step - loss: 0.5841

Epoch 139/5000
10/10 [==============================]10/10 [==============================] - 0s 400us/step - loss: 0.5832

Epoch 140/5000
10/10 [==============================]10/10 [==============================] - 0s 200us/step - loss: 0.5822

Epoch 141/5000
10/10 [==============================]10/10 [==============================] - 0s 301us/step - loss: 0.5813

Epoch 142/5000
10/10 [==============================]10/10 [==============================] - 0s 500us/step - loss: 0.5803

Epoch 1


Epoch 202/5000
10/10 [==============================]10/10 [==============================] - 0s 300us/step - loss: 0.5254

Epoch 203/5000
10/10 [==============================]10/10 [==============================] - 0s 300us/step - loss: 0.5247

Epoch 204/5000
10/10 [==============================]10/10 [==============================] - 0s 300us/step - loss: 0.5239

Epoch 205/5000
10/10 [==============================]10/10 [==============================] - 0s 300us/step - loss: 0.5231

Epoch 206/5000
10/10 [==============================]10/10 [==============================] - 0s 300us/step - loss: 0.5223

Epoch 207/5000
10/10 [==============================]10/10 [==============================] - 0s 200us/step - loss: 0.5215

Epoch 208/5000
10/10 [==============================]10/10 [==============================] - 0s 300us/step - loss: 0.5207

Epoch 209/5000
10/10 [==============================]10/10 [==============================] - 0s 400us/step - loss: 0.5200

Epoch 2


Epoch 269/5000
10/10 [==============================]10/10 [==============================] - 0s 300us/step - loss: 0.4753

Epoch 270/5000
10/10 [==============================]10/10 [==============================] - 0s 200us/step - loss: 0.4745

Epoch 271/5000
10/10 [==============================]10/10 [==============================] - 0s 300us/step - loss: 0.4738

Epoch 272/5000
10/10 [==============================]10/10 [==============================] - 0s 300us/step - loss: 0.4731

Epoch 273/5000
10/10 [==============================]10/10 [==============================] - 0s 300us/step - loss: 0.4724

Epoch 274/5000
10/10 [==============================]10/10 [==============================] - 0s 200us/step - loss: 0.4717

Epoch 275/5000
10/10 [==============================]10/10 [==============================] - 0s 200us/step - loss: 0.4709

Epoch 276/5000
10/10 [==============================]10/10 [==============================] - 0s 200us/step - loss: 0.4702

Epoch 2

...


Epoch 4701/5000
10/10 [==============================]10/10 [==============================] - 0s 200us/step - loss: 0.0158

Epoch 4702/5000
10/10 [==============================]10/10 [==============================] - 0s 300us/step - loss: 0.0158

Epoch 4703/5000
10/10 [==============================]10/10 [==============================] - 0s 400us/step - loss: 0.0158

Epoch 4704/5000
10/10 [==============================]10/10 [==============================] - 0s 200us/step - loss: 0.0158

Epoch 4705/5000
10/10 [==============================]10/10 [==============================] - 0s 300us/step - loss: 0.0157

Epoch 4706/5000
10/10 [==============================]10/10 [==============================] - 0s 200us/step - loss: 0.0157

Epoch 4707/5000
10/10 [==============================]10/10 [==============================] - 0s 200us/step - loss: 0.0157

Epoch 4708/5000
10/10 [==============================]10/10 [==============================] - 0s 300us/step - loss: 0.0157



Epoch 4767/5000
10/10 [==============================]10/10 [==============================] - 0s 300us/step - loss: 0.0153

Epoch 4768/5000
10/10 [==============================]10/10 [==============================] - 0s 400us/step - loss: 0.0153

Epoch 4769/5000
10/10 [==============================]10/10 [==============================] - 0s 300us/step - loss: 0.0153

Epoch 4770/5000
10/10 [==============================]10/10 [==============================] - 0s 300us/step - loss: 0.0153

Epoch 4771/5000
10/10 [==============================]10/10 [==============================] - 0s 201us/step - loss: 0.0152

Epoch 4772/5000
10/10 [==============================]10/10 [==============================] - 0s 200us/step - loss: 0.0152

Epoch 4773/5000
10/10 [==============================]10/10 [==============================] - 0s 302us/step - loss: 0.0152

Epoch 4774/5000
10/10 [==============================]10/10 [==============================] - 0s 200us/step - loss: 0.0152



Epoch 4833/5000
10/10 [==============================]10/10 [==============================] - 0s 600us/step - loss: 0.0148

Epoch 4834/5000
10/10 [==============================]10/10 [==============================] - 0s 300us/step - loss: 0.0148

Epoch 4835/5000
10/10 [==============================]10/10 [==============================] - 0s 401us/step - loss: 0.0148

Epoch 4836/5000
10/10 [==============================]10/10 [==============================] - 0s 399us/step - loss: 0.0148

Epoch 4837/5000
10/10 [==============================]10/10 [==============================] - 0s 599us/step - loss: 0.0148

Epoch 4838/5000
10/10 [==============================]10/10 [==============================] - 0s 200us/step - loss: 0.0148

Epoch 4839/5000
10/10 [==============================]10/10 [==============================] - 0s 200us/step - loss: 0.0148

Epoch 4840/5000
10/10 [==============================]10/10 [==============================] - 0s 601us/step - loss: 0.0147



Epoch 4899/5000
10/10 [==============================]10/10 [==============================] - 0s 300us/step - loss: 0.0143

Epoch 4900/5000
10/10 [==============================]10/10 [==============================] - 0s 200us/step - loss: 0.0143

Epoch 4901/5000
10/10 [==============================]10/10 [==============================] - 0s 300us/step - loss: 0.0143

Epoch 4902/5000
10/10 [==============================]10/10 [==============================] - 0s 300us/step - loss: 0.0143

Epoch 4903/5000
10/10 [==============================]10/10 [==============================] - 0s 300us/step - loss: 0.0143

Epoch 4904/5000
10/10 [==============================]10/10 [==============================] - 0s 300us/step - loss: 0.0143

Epoch 4905/5000
10/10 [==============================]10/10 [==============================] - 0s 200us/step - loss: 0.0143

Epoch 4906/5000
10/10 [==============================]10/10 [==============================] - 0s 300us/step - loss: 0.0143



Epoch 4965/5000
10/10 [==============================]10/10 [==============================] - 0s 200us/step - loss: 0.0139

Epoch 4966/5000
10/10 [==============================]10/10 [==============================] - 0s 200us/step - loss: 0.0139

Epoch 4967/5000
10/10 [==============================]10/10 [==============================] - 0s 300us/step - loss: 0.0139

Epoch 4968/5000
10/10 [==============================]10/10 [==============================] - 0s 200us/step - loss: 0.0139

Epoch 4969/5000
10/10 [==============================]10/10 [==============================] - 0s 200us/step - loss: 0.0139

Epoch 4970/5000
10/10 [==============================]10/10 [==============================] - 0s 300us/step - loss: 0.0139

Epoch 4971/5000
10/10 [==============================]10/10 [==============================] - 0s 200us/step - loss: 0.0138

Epoch 4972/5000
10/10 [==============================]10/10 [==============================] - 0s 300us/step - loss: 0.0138


In [7]:
y_pred = model.predict(X_test)
score = model.evaluate(X_test, y_test)
for idx in range(len(y_pred)):
    print('Label: %f, Prediction: %f' % (y_test[idx], y_pred[idx]))
print('Score: %f' % score)

4/4 [==============================]4/4 [==============================] - 0s 7ms/step

Label: 0.000000, Prediction: 0.041415
Label: 1.000000, Prediction: 0.845193
Label: 0.000000, Prediction: 0.025726
Label: 1.000000, Prediction: 0.998651
Score: 0.059475
